## Read

In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn

In [2]:
df = pd.read_csv(
    r'C:/Users/pontosense/Downloads/Target/ATR_GN_P2-240601-00069-2025-08-27-15-181634-185134.csv',
    index_col=0
)

In [3]:
df['label'] = 2

with open('C:/Users/pontosense/Downloads/Target/ATR_GN_P2-240601-00069-2025-08-27-15-181634-185134.txt', 'r') as fr:
    lines = fr.readlines()
    for i, line in enumerate(lines[:-1]):
        line = line.rstrip('\n').split(',')
        j, l = int(line[0]), int(line[1])
        k = int(lines[i+1].split(',')[0]) - 1
        df.loc[j:k, 'label'] = l

    line = lines[-1].rstrip('\n').split(',')
    j, l = int(line[0]), int(line[1])
    df.loc[j:, 'label'] = 1


In [4]:
df['ln_pts'] = np.log(df['pts']+1)
df['ln_dyn'] = np.log(df['pts_dyn']+1)
df['ln_sta'] = np.log(df['pts']-df['pts_dyn']+1)

df['z_iqr'] = df['z_q3'] - df['z_q1']

In [5]:
df['range'] = np.sqrt(np.square(df['x_c'])+np.square(df['y_c']))

df['dx'] = df['x_c'].diff().fillna(0)
df['dy'] = df['y_c'].diff().fillna(0)
df['dist'] = np.sqrt(np.square(df['dx'])+np.square(df['dy']))

In [6]:
df[['ln_pts', 'z_iqr', 'z_std', 'z_c', 'range', 'l_r', 's_r', 'dist']].describe()

,ln_pts,z_iqr,z_std,z_c,range,l_r,s_r,dist
count,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000,3312.000000
mean,3.757245,0.151343,0.117492,0.703486,1.808366,0.157029,0.073269,0.062353
std,0.750033,0.097444,0.059057,0.164717,0.667683,0.069620,0.025801,0.072770
min,1.791759,0.001326,0.013755,0.116885,0.002175,0.030047,0.004261,0.000000
25%,3.433987,0.087894,0.072488,0.582041,1.257358,0.114481,0.058480,0.021807
50%,3.931826,0.124564,0.108521,0.690462,2.235125,0.139962,0.074673,0.042583
75%,4.248495,0.191632,0.150493,0.804874,2.309790,0.183018,0.089189,0.076200
max,5.545177,0.904304,0.440245,1.357594,3.274375,0.840068,0.294941,1.105550


In [7]:
df.reset_index(inplace=True)

np.random.seed(2025)
torch.manual_seed(2025)

ids = np.random.choice(df.index[:-79], 60, replace=False)

In [8]:
df['label'].value_counts()

label
1    2558
0     754
Name: count, dtype: int64

## Dataset & Dataloader

In [9]:
from torch.utils.data import (
    Dataset,
    DataLoader
)

In [10]:
class PetDataset(Dataset):

    def __init__(self, df, ids):
        self.arr = df[
            ['ln_pts', 'z_iqr', 'z_std', 'z_c', 'range', 'l_r', 's_r', 'dist', 'label']
        ].astype('float32').values
        self.ids = ids

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        jdx = self.ids[idx]
        batch = self.arr[jdx:jdx+80, :-1]
        label = self.arr[jdx:jdx+80, -1]
        return batch, label


In [11]:
train_ds = PetDataset(df, ids)
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True, num_workers=0, drop_last=True)

## Model

In [ ]:
from torchinfo import summary

from pet_binary.layers import (
    EMA,
    Transpose
)

In [ ]:
hs = 64

model = nn.Sequential(
    nn.Linear(8, hs, bias=False),
    Transpose(),
    nn.ReLU(),
    EMA(hs, length=80),
    Transpose(),
    # nn.LayerNorm(hs, 1e-6, False),
    nn.ReLU(),
    nn.Linear(hs, 1, bias=False)
)

summary(model, (2, 80, 8), col_names=['output_size', 'num_params'])

Layer (type:depth-idx)                   Output Shape              Param #
Sequential                               [2, 80, 1]                --
├─Linear: 1-1                            [2, 80, 64]               512
├─ReLU: 1-2                              [2, 80, 64]               --
├─Transpose: 1-3                         [2, 64, 80]               --
├─EMA: 1-4                               [2, 64, 80]               128
├─Transpose: 1-5                         [2, 80, 64]               --
├─ReLU: 1-6                              [2, 80, 64]               --
├─Linear: 1-7                            [2, 80, 1]                64
Total params: 704
Trainable params: 704
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.01
Forward/backward pass size (MB): 0.17
Params size (MB): 0.00
Estimated Total Size (MB): 0.17

## Train

In [103]:
from torchmetrics import MetricCollection
from torchmetrics.classification import (
    BinaryAccuracy,
    BinaryAUROC
)

In [104]:
# criterion
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)

metric = MetricCollection({'acc':BinaryAccuracy(), 'auc':BinaryAUROC()})

In [105]:
%%time

epochs = 21
step = 0

model.train()

for epoch in range(epochs):

    for batch, label in train_dl:
        logit = model(batch).squeeze(-1)
        loss = loss_fn(logit, label)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        proba = torch.sigmoid(logit)
        metric.update(proba, label)
        step += 1

    res = metric.compute()
    print(f'epoch {epoch} - acc: {res["acc"]:.4f} - auc: {res["auc"]:.4f}')
    metric.reset()


epoch 0 - acc: 0.6951 - auc: 0.5858
epoch 1 - acc: 0.6775 - auc: 0.7572
epoch 2 - acc: 0.6978 - auc: 0.7964
epoch 3 - acc: 0.6761 - auc: 0.8200
epoch 4 - acc: 0.6938 - auc: 0.8284
epoch 5 - acc: 0.6786 - auc: 0.8680
epoch 6 - acc: 0.6938 - auc: 0.8785
epoch 7 - acc: 0.6786 - auc: 0.8764
epoch 8 - acc: 0.6759 - auc: 0.9040
epoch 9 - acc: 0.6761 - auc: 0.8995
epoch 10 - acc: 0.7116 - auc: 0.9159
epoch 11 - acc: 0.6964 - auc: 0.9042
epoch 12 - acc: 0.6580 - auc: 0.9222
epoch 13 - acc: 0.6674 - auc: 0.9231
epoch 14 - acc: 0.6772 - auc: 0.9348
epoch 15 - acc: 0.6759 - auc: 0.9395
epoch 16 - acc: 0.6772 - auc: 0.9469
epoch 17 - acc: 0.6900 - auc: 0.9573
epoch 18 - acc: 0.7295 - auc: 0.9512
epoch 19 - acc: 0.7563 - auc: 0.9565
epoch 20 - acc: 0.7993 - auc: 0.9610
CPU times: total: 4.22 s
Wall time: 540 ms


In [106]:
model

Sequential(
  (0): Linear(in_features=8, out_features=64, bias=False)
  (1): ReLU()
  (2): Transpose()
  (3): EMA(in_chn=64, length=80)
  (4): Transpose()
  (5): ReLU()
  (6): Linear(in_features=64, out_features=1, bias=False)
)

In [107]:
model[3].params.clip(0, 1).flatten()

tensor([0.8896, 0.3408, 0.8271, 0.2732, 0.6323, 0.1168, 0.0968, 0.4643, 0.1605,
        0.0391, 0.1931, 0.8049, 0.8968, 0.3196, 0.8693, 0.9878, 0.3313, 0.2141,
        0.7117, 0.6708, 0.7730, 0.8281, 0.6684, 0.4420, 0.5452, 0.8477, 0.5310,
        0.4659, 0.4535, 0.1802, 0.7349, 0.2970, 0.4297, 0.7034, 1.0000, 0.9665,
        0.4526, 0.0699, 0.5022, 0.6483, 0.6722, 0.4433, 0.6671, 0.6215, 0.0000,
        0.4187, 0.6780, 0.6357, 0.4093, 0.1970, 0.3271, 0.8704, 0.5436, 0.7217,
        0.8717, 0.9095, 0.6979, 0.6045, 0.6304, 0.2393, 0.3845, 1.0000, 0.1757,
        0.2430], grad_fn=<ViewBackward0>)

In [108]:
model[3].x0.clip(0)

tensor([[0.4583, 0.1109, 0.0563, 0.0526, 0.1514, 0.6142, 0.7008, 0.1118, 0.7139,
         0.4707, 0.5919, 0.8427, 0.0505, 0.4375, 0.0000, 0.4740, 1.0324, 0.9368,
         0.1332, 0.7751, 0.2168, 0.3069, 0.5121, 0.4555, 0.1735, 0.8811, 0.8224,
         0.8982, 0.5241, 0.0000, 0.5818, 0.9138, 0.8097, 0.1720, 0.0152, 0.9100,
         0.0241, 0.0801, 0.5000, 0.3832, 0.3759, 0.7161, 0.2373, 0.5897, 0.3084,
         0.4604, 0.5902, 0.8673, 0.4089, 0.0000, 0.8516, 0.3164, 0.5932, 0.1215,
         0.3152, 0.7942, 0.0754, 0.4454, 0.3410, 0.6171, 0.0000, 0.4453, 0.1170,
         0.4639]], grad_fn=<ClampBackward1>)